In [1]:
!pip install pycoingecko

In [2]:
import requests
import pandas as pd

def get_crypto_data(crypto_id):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}"
    result = requests.get(url)
    data = result.json()

    market_data = data['market_data']

    market_cap = market_data['market_cap']['usd']
    total_volume = market_data['total_volume']['usd']
    price_change_percentage_24h = market_data['price_change_percentage_24h']

    return market_cap, total_volume, price_change_percentage_24h

# Get data for Bitcoin
market_cap, total_volume, price_change_percentage_24h = get_crypto_data('bitcoin')

# Create a DataFrame
df = pd.DataFrame({
    'Market Cap': [market_cap],
    'Total Volume': [total_volume],
    'Price Change Percentage (24h)': [price_change_percentage_24h]
})

print(df)


     Market Cap  Total Volume  Price Change Percentage (24h)
0  747062528377   25707209692                        2.11816


In [10]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime

# Initialize CoinGeckoAPI
cg = CoinGeckoAPI()

# Define your cryptocurrency and currency
crypto_id = 'bitcoin'  # Cryptocurrency ID (e.g., 'bitcoin')
vs_currency = 'usd'  # Currency (e.g., 'usd')

# Fetch historical data for the last 5 years
historical_data = cg.get_coin_market_chart_by_id(id=crypto_id, vs_currency=vs_currency, days='1825')

# Extract and format the relevant data
dates = [datetime.fromtimestamp(item[0]/1000).date() for item in historical_data['prices']]
prices = [item[1] for item in historical_data['prices']]

market_caps = [mc for _, mc in historical_data['market_caps']]
total_volumes = [vol for _, vol in historical_data['total_volumes']]


# Create a DataFrame
df = pd.DataFrame({
    'Date': pd.to_datetime(dates), #convert the 'Date' column to a DatetimeIndex before resampling.
    'Price': prices,
    'Market Cap': market_caps,
    'Total Volume': total_volumes
})

# Sample the data every three months
df = df.resample('3M', on='Date').mean()

# Display the DataFrame
df

,Price,Market Cap,Total Volume
Date,,,
2018-12-31,3684.055037,6.421284e+10,3.493655e+09
2019-03-31,3758.464113,6.592411e+10,6.015345e+09
2019-06-30,7214.372806,1.278302e+11,2.377031e+10
2019-09-30,10400.709391,1.858836e+11,2.408472e+10
2019-12-31,7996.008607,1.442994e+11,2.385612e+10
2020-03-31,8273.873759,1.506556e+11,4.227246e+10
2020-06-30,8622.948719,1.584174e+11,3.373422e+10
2020-09-30,10606.689985,1.958534e+11,2.308773e+10
2020-12-31,16648.682657,3.088814e+11,2.759524e+10


# Assignment: Better Investment

Could you create a python script that retrieves and analyses the quarterly performance of Bitcoin and Ethereum against the GBP from January 2018 to January 2023? Additionally, could you ascertain which cryptocurrency, Bitcoin or Ethereum, would have yielded a higher return on investment by January 2023?



In [16]:
from pycoingecko import CoinGeckoAPI
import pandas as pd

def get_crypto_data(crypto_id, vs_currency, start_date, end_date):
    cg = CoinGeckoAPI()
    historical_data = cg.get_coin_market_chart_range_by_id(id=crypto_id, vs_currency=vs_currency, from_timestamp=start_date, to_timestamp=end_date)
    prices = historical_data['prices']
    return [(int(item[0]), item[1]) for item in prices]

def process_data(data):
    df = pd.DataFrame(data, columns=['Timestamp', 'Price'])
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    return df['Price'].resample('Q').mean()

# Timestamps for January 2018 and January 2023
start_date = int(pd.Timestamp('2018-01-01').timestamp())
end_date = int(pd.Timestamp('2023-01-01').timestamp())

# Retrieve data
bitcoin_data = get_crypto_data('bitcoin', 'gbp', start_date, end_date)
ethereum_data = get_crypto_data('ethereum', 'gbp', start_date, end_date)

# Process data
bitcoin_prices = process_data(bitcoin_data)
ethereum_prices = process_data(ethereum_data)

# Calculate ROI
bitcoin_roi = (bitcoin_prices.iloc[-1] - bitcoin_prices.iloc[0]) / bitcoin_prices.iloc[0]
ethereum_roi = (ethereum_prices.iloc[-1] - ethereum_prices.iloc[0]) / ethereum_prices.iloc[0]

# print("Bitcoin ROI:", bitcoin_roi)
# print("Ethereum ROI:", ethereum_roi)

# Determine which had higher ROI
if bitcoin_roi > ethereum_roi:
    if bitcoin_roi>=1:
        print(f"Bitcoin had a higher ROI, a person who invested 1 GBP, will get {bitcoin_roi} on his 1 GBP")
    else:
        print(f"Bitcoin had a higher ROI, but that investor had incurred a loss. That who invested will have lost {(1- bitcoin_roi)*100:.2f}% of his/her investment")
else:
    print("Ethereum had a higher ROI")


Bitcoin had a higher ROI, but that investor had incurred a loss. That who invested will have lost 22.67% of his/her investment
